# Trabalho de Aprendizado de Máquina 2

Ana Ellen Deodato P Silva - 800206  
Vinicius de Oliveira Guimarães - 802431  
Vinícius Gonçalves Perillo - 800219  

In [ ]:
#!wget https://nlp.stanford.edu/data/glove.6B.zip; unzip glove.6B.zip

# Base de dados

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_train = pd.read_csv('/content/DBPEDIA_train.csv')
data_train.info()

data_test = pd.read_csv("/content/DBPEDIA_test.csv")
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240942 entries, 0 to 240941
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    240942 non-null  object
 1   l1      240942 non-null  object
 2   l2      240942 non-null  object
 3   l3      240942 non-null  object
dtypes: object(4)
memory usage: 7.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60794 entries, 0 to 60793
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    60794 non-null  object
 1   l1      60794 non-null  object
 2   l2      60794 non-null  object
 3   l3      60794 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [ ]:
data_train.isna().mean().round(4).mul(100)

text    0.0
l1      0.0
l2      0.0
l3      0.0
dtype: float64

In [ ]:
data_train.sample(10)

,text,l1,l2,l3
79542,Olga Sergeyevna Kapranova (Russian: Ольга Серг...,Agent,Athlete,Gymnast
211614,The Roman Catholic Diocese of Charleston is an...,Place,ClericalAdministrativeRegion,Diocese
226528,The Mulholland Dam is a Los Angeles Department...,Place,Infrastructure,Dam
39887,Nebuchadnezzar II (/ˌnɛbjᵿkədˈnɛzər/; Aramaic:...,Agent,Person,Monarch
75254,The Taylor Correctional Institution is a state...,Place,Building,Prison
23442,Ruth Osborne (previously Benson) is a fictiona...,Agent,FictionalCharacter,SoapCharacter
220878,"Jim Paek (Korean: 백지선, Baek Chi-sun, born Apri...",Agent,WinterSportPlayer,IceHockeyPlayer
226153,Henry Scott Riddell (23 September 1798 – 1870)...,Agent,Writer,Poet
10253,"The 1909 U.S. Open was the 15th U.S. Open, hel...",Event,Tournament,GolfTournament
139142,Vividred Operation (ビビッドレッド・オペレーション Bibiddored...,Work,Comic,Manga


In [ ]:
data_train['l1'].value_counts()

Agent             124798
Place              45877
Species            21472
Work               21013
Event              19106
SportsSeason        5883
UnitOfWork          1761
TopicalConcept       784
Device               248
Name: l1, dtype: int64

In [ ]:
df = data_train['l2'].value_counts()

In [ ]:
data_train['l3'].value_counts()

AcademicJournal          1924
Manga                    1924
FigureSkater             1924
OlympicEvent             1923
SoccerTournament         1922
                         ... 
Cycad                     145
AnimangaCharacter         144
BeachVolleyballPlayer     137
CanadianFootballTeam      133
BiologicalDatabase        129
Name: l3, Length: 219, dtype: int64

# Processamento da base de dados + PLN

In [ ]:
import nltk
import spacy

In [ ]:
data_train = data_train.assign(sep='train')
data_test = data_test.assign(sep='test')

data = pd.concat([data_train, data_test])

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('glove.6B')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Error loading glove.6B: Package 'glove.6B' not found in
[nltk_data]     index


False

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
def process_text(sentence):
    global stop_words

    tokens = nltk.tokenize.word_tokenize(sentence, language= 'english')
    return [lemmatizer.lemmatize(word) for word in tokens if word.lower().strip() not in stop_words]

In [ ]:
# Aqui estamos pegando o contéudo da coluna 'text' que é o texto bruto.
# Em seguida, dentro da função process_text, iremos pegar essa sentença e
# separar todos os tokens e deixar apenas aqueles que não são stop_words do
# idioma inglês

data['tokens'] = data['text'].apply(process_text)
data

,text,l1,l2,l3,sep,tokens
0,"William Alexander Massey (October 7, 1856 – Ma...",Agent,Politician,Senator,train,"[William, Alexander, Massey, (, October, 7, ,,..."
1,Lions is the sixth studio album by American ro...,Work,MusicalWork,Album,train,"[Lions, sixth, studio, album, American, rock, ..."
2,"Pirqa (Aymara and Quechua for wall, hispaniciz...",Place,NaturalPlace,Mountain,train,"[Pirqa, (, Aymara, Quechua, wall, ,, hispanici..."
3,Cancer Prevention Research is a biweekly peer-...,Work,PeriodicalLiterature,AcademicJournal,train,"[Cancer, Prevention, Research, biweekly, peer-..."
4,The Princeton University Chapel is located on ...,Place,Building,HistoricBuilding,train,"[Princeton, University, Chapel, located, unive..."
...,...,...,...,...,...,...
60789,Darwin Smith was CEO of Kimberly-Clark from 19...,Agent,Person,BusinessPerson,test,"[Darwin, Smith, CEO, Kimberly-Clark, 1971, 199..."
60790,The HKRU Premiership is the main domestic comp...,Agent,SportsLeague,SoccerLeague,test,"[HKRU, Premiership, main, domestic, competitio..."
60791,"Leo Harvey Diegel (April 20, 1899 – May 5, 195...",Agent,Athlete,GolfPlayer,test,"[Leo, Harvey, Diegel, (, April, 20, ,, 1899, –..."
60792,"Clement Reginald \""Clem\"" Splatt (17 September...",Agent,Athlete,AustralianRulesFootballPlayer,test,"[Clement, Reginald, \, '', Clem\, '', Splatt, ..."


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

# Mudar de 50d para 300d
glove_file = '/content/glove.6B.50d.txt'
word2vec_file = '/content/glove.6B.50d.word2vec'

glove2word2vec(glove_file, word2vec_file)

word_vectors = KeyedVectors.load_word2vec_format(word2vec_file, binary=False)


<ipython-input-15-22e432d2a304>:7: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_file)


In [ ]:
def word2vec_mean(tokens):
    vectors = []

    for t in tokens:
        try:
            vectors.append(word_vectors[t.lower()])
        except KeyError:
            print(f'{t.lower()} not present')
    #print(np.mean(vectors, axis= 0))
    return np.mean(vectors, axis= 0)


In [ ]:
train_data = data[data['sep'] == 'train']['tokens'].apply(word2vec_mean).to_list()
test_data = data[data['sep'] == 'test']['tokens'].apply(word2vec_mean).to_list()

A saída de streaming foi truncada nas últimas 5000 linhas.
proudia/dignity not present
kiarina not present
kordela not present
αγλαΐα not present
κιαρίνα not present
κορδέλα not present
kordela not present
foucauldian not present
pheugopedius not present
eisenmanni not present
הגביע not present
הארץ-ישראלי‎‎ not present
hagavia not present
haeretz-israeli not present
eifa not present
vermilion—each not present
1972–80 not present
1980–87 not present
afield.the not present
sohodolls not present
stripper\ not present
cool\ not present
stripper\ not present
vampire\ not present
generation\ not present
posford not present
shpongled not present
posford not present
23\ not present
2ॐ\ not present
ॐ\ not present
4.385 not present
malaren not present
varado not present
traffic-congestion not present
kandivli not present
bandra-worli not present
huanaco not present
amarillae not present
pachitea not present
isbrücker not present
thirtysixtwentyfive\ not present
thirtysixtwentyfive\ not present


In [ ]:
pd.DataFrame(train_data).to_csv('train_data.csv')
pd.DataFrame(test_data).to_csv('test_data.csv')

In [ ]:
train = data[data['sep'] == 'train']
test = data[data['sep'] == 'test']

y_train = train[["l1", "l2", "l3"]].to_numpy()
y_test = test[["l1", "l2", "l3"]].to_numpy()

# Modelo Hierárquico

### Logistical regression

In [ ]:
#!pip install hiclass

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import time

### Definimos um classificador para utilizar como base na nossa estratégia.
### Neste caso, usaremos a regressão logística.
base_classifier = LogisticRegression(
    random_state=42,
    max_iter=10000,
    n_jobs=1,
)

In [ ]:
### Importamos do HiClass o módulo LocalClassifierPerParentNode que corresponde a estratégia desejada
from hiclass import LocalClassifierPerParentNode

###Instaciamos a estratégia de Classificador local por nó pai.
lcppn = LocalClassifierPerParentNode(
    local_classifier=base_classifier, ##Indicamos quem é nosso classificador base.
    verbose=0,
    n_jobs=1,
)

### Criamos um fluxo de execução com a classe Pipeline do sklearn.
pipeline = Pipeline([
    ('model', lcppn), ### Modelo de classificação por nó pai.
])

In [ ]:
beginning = time.time()
pipeline.fit(train_data, y_train) ### Treinamento do modelo usando o fluxo definido.
end = time.time()

In [ ]:
### Imprimindo tempo de treinamento.
print(f"Tempo de treinamento: {end - beginning}")

Tempo de treinamento: 537.781806230545


In [ ]:
### Realizando predições

predictions = pipeline.predict(test_data)

In [ ]:
comp_log = pd.concat([pd.DataFrame(predictions), pd.DataFrame(y_test)], axis =1).sort_index(axis=1)
comp_log

,0,0,1,1,2,2
0,Agent,Agent,Person,Politician,OfficeHolder,PrimeMinister
1,Agent,Agent,Actor,Actor,AdultActor,AdultActor
2,Place,Place,Infrastructure,Infrastructure,Dam,Dam
3,Event,Event,Race,Race,CyclingRace,CyclingRace
4,Agent,Agent,Athlete,Athlete,MartialArtist,MartialArtist
...,...,...,...,...,...,...
60789,Agent,Agent,Person,Person,Judge,BusinessPerson
60790,Agent,Agent,SportsLeague,SportsLeague,RugbyLeague,SoccerLeague
60791,Agent,Agent,Athlete,Athlete,GolfPlayer,GolfPlayer
60792,Agent,Agent,Athlete,Athlete,AustralianRulesFootballPlayer,AustralianRulesFootballPlayer


In [ ]:
pl50 = pd.DataFrame(predictions)
pl50

,0,1,2
0,Agent,Person,OfficeHolder
1,Agent,Actor,AdultActor
2,Place,Infrastructure,Dam
3,Event,Race,CyclingRace
4,Agent,Athlete,MartialArtist
...,...,...,...
60789,Agent,Person,Judge
60790,Agent,SportsLeague,RugbyLeague
60791,Agent,Athlete,GolfPlayer
60792,Agent,Athlete,AustralianRulesFootballPlayer


In [ ]:
# Computando métrica de avaliação

from hiclass.metrics import f1
from sklearn.metrics import accuracy_score

f1_hierarchical_log = f1(y_test, predictions)
print(f"f-score: {f1_hierarchical_log}\n")

f-score: 0.7959063942713644



In [ ]:
test_data

### Random Forest

In [ ]:
!pip install hiclass

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

import time

### Definimos um classificador para utilizar como base na nossa estratégia.
### Neste caso, usaremos a regressão logística.
base_classifier = RandomForestClassifier(
    random_state=42,
    n_jobs=1,
)

In [ ]:
### Importamos do HiClass o módulo LocalClassifierPerParentNode que corresponde a estratégia desejada
from hiclass import LocalClassifierPerParentNode

###Instaciamos a estratégia de Classificador local por nó pai.
lcppn = LocalClassifierPerParentNode(
    local_classifier=base_classifier, ##Indicamos quem é nosso classificador base.
    verbose=0,
    n_jobs=1,
)

### Criamos um fluxo de execução com a classe Pipeline do sklearn.
pipeline = Pipeline([
    ('model', lcppn), ### Modelo de classificação por nó pai.
])

In [ ]:
beginning = time.time()
pipeline.fit(train_data, y_train) ### Treinamento do modelo usando o fluxo definido.
end = time.time()

In [ ]:
### Imprimindo tempo de treinamento.
print(f"Tempo de treinamento: {end - beginning}")

Tempo de treinamento: 1121.7102012634277


In [ ]:
### Realizando predições

predictions_forest = pipeline.predict(test_data)
predictions_forest

array([['Agent', 'Athlete', 'MartialArtist'],
       ['Agent', 'Actor', 'AdultActor'],
       ['Place', 'Infrastructure', 'Dam'],
       ...,
       ['Agent', 'Athlete', 'GolfPlayer'],
       ['Agent', 'Athlete', 'AustralianRulesFootballPlayer'],
       ['Place', 'Building', 'Castle']], dtype='<U104')

In [ ]:
pd.concat([pd.DataFrame(predictions_forest), pd.DataFrame(y_test)], axis =1).sort_index(axis=1)

,0,0,1,1,2,2
0,Agent,Agent,Athlete,Politician,MartialArtist,PrimeMinister
1,Agent,Agent,Actor,Actor,AdultActor,AdultActor
2,Place,Place,Infrastructure,Infrastructure,Dam,Dam
3,Event,Event,Race,Race,CyclingRace,CyclingRace
4,Agent,Agent,Athlete,Athlete,MartialArtist,MartialArtist
...,...,...,...,...,...,...
60789,Agent,Agent,Person,Person,Philosopher,BusinessPerson
60790,Agent,Agent,SportsTeam,SportsLeague,RugbyClub,SoccerLeague
60791,Agent,Agent,Athlete,Athlete,GolfPlayer,GolfPlayer
60792,Agent,Agent,Athlete,Athlete,AustralianRulesFootballPlayer,AustralianRulesFootballPlayer


In [ ]:
# Computando métrica de avaliação

from hiclass.metrics import f1
from sklearn.metrics import accuracy_score

f1_hierarchical_forest = f1(y_test, predictions_forest)
print(f"f-score: {f1_hierarchical_forest}\n")

#ccuracy_forest = accuracy_score(y_test, predictions_forest)#
#rint(f'Acurácia: {accuracy_forest}')

f-score: 0.8398252020484478



In [ ]:
pf50 = pd.DataFrame(predictions_forest)
pf50

,0,1,2
0,Agent,Athlete,MartialArtist
1,Agent,Actor,AdultActor
2,Place,Infrastructure,Dam
3,Event,Race,CyclingRace
4,Agent,Athlete,MartialArtist
...,...,...,...
60789,Agent,Person,Philosopher
60790,Agent,SportsTeam,RugbyClub
60791,Agent,Athlete,GolfPlayer
60792,Agent,Athlete,AustralianRulesFootballPlayer


In [ ]:
p.to_csv('Predictions_forest_50.csv')

# Métricas

In [ ]:
from hiclass.metrics import f1
from hiclass.metrics import precision
from hiclass.metrics import recall


pf300 = pd.read_csv("/content/Predictions_forest.csv")
pl300 = pd.read_csv("/content/Predictions_Logistica (1).csv")
#pf50 = pd.read_csv("/content/Predictions_Forest_50.csv")
#pl50 = pd.read_csv("/content/Predictions_Logistical_50.csv")

f1_pf50 = f1(y_test, pf50)
print(f"f-score pf50: {f1_pf50}\n")
f1_pl50 = f1(y_test, pl50)
print(f"f-score pl50: {f1_pl50}\n")
f1_pf300 = f1(y_test, pf300)
print(f"f-score pf300: {f1_pf300}\n")
f1_pl300 = f1(y_test, pl300)
print(f"f-score pl300: {f1_pl300}\n")

precision_pf50 = precision(y_test, pf50)
print(f"precision pf50: {precision_pf50}\n")
precision_pl50 = precision(y_test, pl50)
print(f"precision pl50: {precision_pl50}\n")
precision_pf300 = precision(y_test, pf300)
print(f"precision pf300: {precision_pf300}\n")
precision_pl300 = precision(y_test, pl300)
print(f"precision pl300: {precision_pl300}\n")

recall_pf50 = recall(y_test, pf50)
print(f"recall pf50: {recall_pf50}\n")
recall_pl50 = recall(y_test, pl50)
print(f"recall pl50: {recall_pl50}\n")
recall_pf300 = recall(y_test, pf300)
print(f"recall pf300: {recall_pf300}\n")
recall_pl300 = recall(y_test, pl300)
print(f"recall pl300: {recall_pl300}\n")

f-score pf50: 0.8398252020484478

f-score pl50: 0.7959063942713644

f-score pf300: 0.7445189609876915

f-score pl300: 0.7786952659801953

precision pf50: 0.8398252020484478

precision pl50: 0.7959063942713644

precision pf300: 0.65145409086423

precision pl300: 0.681358357732671

recall pf50: 0.8398252020484478

recall pl50: 0.7959063942713644

recall pf300: 0.8686054544856401

recall pl300: 0.908477810310228

